In [5]:
#PYTHON NOTEBOOK SCRIPT TO GET GENBANK SEQUENCES OF INTEREST AND WRITE FASTA FILE

from Bio import Entrez, SeqIO
Entrez.email = 'curiousgeorge@ufl.edu'

#prompt user for organism and number of results desired
query= input('Enter organism name')
usermax=input('Enter maximum number of results ')

#send query to Entrez nucleotide database
handle = Entrez.esearch(db='nucleotide', term = query, field='organism', retmax=usermax)
record = Entrez.read(handle)
handle.close()

#get the 'IDList' field from the results
ids=(record["IdList"])

#function to get sequences from the genbank IDs returned from the search and write to 'query.fasta'
def get_sequences():
    open('query.fasta','w')
    for seq_id in ids:
        handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text",retmax=1)
        f=open('query.fasta','a') #a = append mode
        f.write(handle.read()) #add data to end of file
get_sequences()
handle.close()

Enter organism name inocybe
Enter maximum number of results  60


In [3]:
#PART 2 - getting only sequences for a gene of interest... 

#make empty list to stash the search results in (to eventually search through with 'if' statment)
my_seqlist = []
for seq_record in SeqIO.parse("query.fasta", "fasta"):
    my_seqlist.append(seq_record)
    my_seqlist[0]

#prompt user for second query (gene or metadata)
gene=input("Specify gene (or other metadata) of interest: ")

#search through my_seqlist for 'gene', and append those records to a new file called 'second.fasta'
with open("second.fasta", "w") as handle: #first open an empty, writable file
    for seq_record in my_seqlist:
        if gene in seq_record.description: #every time 'gene' is encountered in the 'description' attribute of the record
            SeqIO.write(seq_record, handle, "fasta") #write that to the file

Enter Gene of Interest:  ITS


In [4]:
#Code adapted from Biopython help forums and adapted from work in the Biopython Cookbook Change et al 2020

from Bio import SeqIO
from Bio.SeqUtils.CheckSum import seguid

#function to check for and remove duplicate records
def remove_duplicates(records):
    check_sequences = set()
    for record in records:
        checksum = seguid(record.seq)
        if checksum in check_sequences:
            print("Removing duplicates %s into outfile" % record.id ) 
            continue
        check_sequences.add(checksum)
        yield record
        if record.id == record.seq:
            print("No duplicates!")

saved_sequences = remove_duplicates(SeqIO.parse("second.fasta", "fasta")) #run remove duplicates on second.fasta
count = SeqIO.write(saved_sequences, "final.fasta", "fasta")
print ("Final number of sequences %s" %count)

#read the new file and display the gene ids

print("Unique designations are: ")
for seq_record in SeqIO.parse("final.fasta", "fasta"):
    print(seq_record.id) #print only the id field 

Number of remaining sequence 2
Unique designations are: 
NR_168766.1
NR_168765.1
